# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-09 15:28:41] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-09 15:28:41] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-09 15:28:41] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-09 15:28:44] INFO server_args.py:1622: Attention backend not specified. Use fa3 backend by default.


[2026-01-09 15:28:44] INFO server_args.py:2519: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=120 avail_mem=13.04 GB):   5%|▌         | 1/20 [00:00<00:06,  2.94it/s]

Capturing batches (bs=72 avail_mem=13.02 GB):  25%|██▌       | 5/20 [00:00<00:01,  9.15it/s]

Capturing batches (bs=32 avail_mem=12.99 GB):  55%|█████▌    | 11/20 [00:00<00:00, 17.10it/s]

Capturing batches (bs=8 avail_mem=12.97 GB):  70%|███████   | 14/20 [00:01<00:00, 18.90it/s] 

Capturing batches (bs=1 avail_mem=12.96 GB): 100%|██████████| 20/20 [00:01<00:00, 15.15it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  a***.  I have had a long life. I was born with my parents' fingerprints on my hands and every day I have to tell them a few things. 

Is it true that the person I am is my own twin? If so, can you explain your reasoning for naming me in this way? No, I am not a double, I am a twin. When I was born, my parents had double fingerprints on their hands. My fingerprints had the same structure as my parents', but because of the alignment of my DNA, my hands were unique. This made my fingerprints unique to me.

Some may wonder why I am
Prompt: The president of the United States is
Generated text:  a person. (Judge true or false.)
To determine whether the statement "The president of the United States is a person" is true or false, let's break down the information given and analyze it step by step.

1. Identify the subject of the statement: The president of the United States.
2. Identify the characteristic of the subject: Being a person.
3. Determine th

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a short, interesting fact about yourself]. I'm always looking for new challenges and opportunities to grow and learn. What's your favorite hobby or activity? I enjoy [insert a short, interesting fact about your hobby or activity]. I'm always looking for ways to expand my knowledge and skills. What's your favorite book or movie? I love [insert a short, interesting fact about your favorite book or movie]. I'm always looking for

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is the largest city in France and the second-largest city in the European Union. It is also the oldest capital city in Europe, having been the seat of the French monarchy since the 12th century. Paris is known for its rich history, art, and culture, including the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. The city is also home to many famous landmarks and attractions, including the Louvre, the Arc de Triomphe, and the Champs-Élysées. Paris is a major transportation hub, with the Eiffel Tower serving as a symbol of the city

Prompt: Explain possible future trends in artificial intelligence. The future of AI is
Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: As AI becomes more advanced, it is likely to be integrated with human intelligence in new and innovative ways. This could include the development of AI that can better understand and adapt to human emotions and behaviors, or t

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], and I am a [age] year-old AI system designed to assist individuals in their daily lives. I'm here to help, answer questions, and offer helpful advice to anyone who needs it. I'm a member of the [AI system name], a team of highly trained professionals who specialize in providing accurate and helpful responses to user queries. I'm always here to assist you and help you achieve your goals. What can I do for you today? Let me know! [End of self-introduction] Here is a more formal and neutral self-introduction for a fictional character. 

Hello, my name is [Your

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. It is a city located in the heart of the French countryside and is the country's largest and most populous city. The city is known for its rich history, beautiful architecture, and vibrant

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

 seamless

 and

 interconnected

 world

,

In [6]:
llm.shutdown()